Change point
1. Use Cross varidation
2. Decrease size of code
---accuracy 0.76794---

In [1]:
import numpy as np
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [2]:
train_x = train.drop(['Survived'],axis=1)
train_y = train['Survived']
data = pd.concat([train_x,test],sort=False)
data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
data['Sex'].replace(['male','female'],[0,1],inplace=True)
data['Embarked'].fillna(('S'),inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)

In [3]:
train_x = data[:len(train)]
test_x = data[len(train):]

In [4]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold

In [5]:
#List to save score of each fold
scores_accuracy = []
scores_logloss = []
#Do cross varidation???
kf = KFold(n_splits=4,shuffle=True,random_state=71)

In [6]:
from xgboost import XGBClassifier

In [8]:
for tr_idx,va_idx in kf.split(train_x):
    #iloc can select row and column(0~).
    tr_x ,va_x = train_x.iloc[tr_idx],train_x.iloc[va_idx]
    tr_y ,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]
    
    model = XGBClassifier(n_estimators=20,random_state=71)
    model.fit(tr_x,tr_y)
    
    #Output predict value as %.
    va_pred = model.predict_proba(va_x)[:,1]
    logloss = log_loss(va_y,va_pred)
    accuracy = accuracy_score(va_y,va_pred>0.5)
    
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)
    

In [9]:
#Output mean of each fold score.
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss:{logloss:.4f},accuracy:{accuracy:.4f}')

logloss:0.4358,accuracy:0.8204


In [13]:
pred = model.predict_proba(test_x)[:,1]

In [15]:
pred_label = np.where(pred>0.5,1,0)
pred_label[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0])

In [16]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred_label})

In [18]:
submission.to_csv('submission20201104.csv',index=False)